In [1]:
from bs4 import BeautifulSoup
import urllib.request
import re
import nltk
from string import punctuation
from nltk.corpus import stopwords
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from string import punctuation
import string
import collections

In [52]:
#get the word list from url
def UrlToWords(url):
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:23.0) Gecko/20100101 Firefox/23.0'}
    req = urllib.request.Request(url=url, headers=headers)
    try:
        html = urllib.request.urlopen(req).read()
    except:
        return 
    else:
        soup = BeautifulSoup(html, "lxml")
        st = LancasterStemmer()
        [s.extract() for s in soup('script')]
        [s.extract() for s in soup('style')]
        t_string = soup.get_text()
        tokens = nltk.word_tokenize(t_string)
        text = [w.lower() for w in tokens]

        all_p = punctuation + '--' + '|' +'-' + '~' + '``' + '"' + '=' + '_' + '\"'+ '/'
        words = []
        for i in text:
            i = re.sub('\s+', '', str(i))
            i = re.sub('\\\\|\^|\_', '' , i)
            i = re.sub('[^A-Za-z]', '', i)
            if i != '' and len(i) <= 15  and i not in all_p:
                words.append(i)

        POS_tag = nltk.pos_tag(words)
        wordnet_lemmatizer = WordNetLemmatizer()
        adjective_tags = ['JJ','JJR','JJS']
        #根据词性进行词根还原
        lemmatized_text = []
        for word in POS_tag:
            if word[1] in adjective_tags:
                lemmatized_text.append(str(wordnet_lemmatizer.lemmatize(word[0],pos="a")))
            else:
                lemmatized_text.append(str(wordnet_lemmatizer.lemmatize(word[0]))) #default POS = noun
        words = lemmatized_text
        POS_tag = nltk.pos_tag(words)

        stopwords2 = []
        wanted_POS = ['NN','NNS','NNP','NNPS','JJ','JJR','JJS','VBG','FW'] 
        for word in POS_tag:
            if word[1] not in wanted_POS:
                stopwords2.append(word[0])

        punctuations = list(str(string.punctuation))
        stopwords2 = stopwords2 + punctuations
        stop = stopwords.words('english')
        stopwords_plus = stopwords2 + stop
        stopwords_plus = set(stopwords_plus)

        processed_text = []
        for word in words:
            if word not in stopwords_plus:
                processed_text.append(word)

        words = processed_text
        words = [w for w in words if len(w)>2]
        #words = [w for w in words if w not in all_p]
        fre = collections.Counter(words)
        #fre = fre.most_common()
        fre = dict(fre)
        t1 = []
        t2 = []
        for k,v in fre.items():
            if v==1:
                t1.append(k)
        #frequences = dict(zip(t1,t2))
        words = [w for w in words if w not in t1]
        return words

In [69]:
#url = 'http://www.skysports.com/football/news/11095/11464517/thibaut-courtois-eden-hazard-toby-alderweireld-premier-leagues-belgian-stars-on-the-move'
f = open('url_list.txt', 'r')
user_url = f.read().splitlines()
f.close()
user_data = []
for url in user_url:
    print(url)
    words = UrlToWords(url)
    if words:
        words = list(set(words))
        user_data.extend(words)

http://www.baidu.com/
http://www.cnblogs.com/pinard/p/6244265.html
http://www.cnblogs.com/pinard/p/6239403.html
http://blog.csdn.net/zengxiantao1994/article/details/72787849
http://blog.csdn.net/gao1440156051/article/details/44162269
https://my.oschina.net/findbill/blog/535044
http://www.ifis.uni-luebeck.de/~moeller/Lectures/WS-16-17/Web-Mining-Agents/PCA-SVD.pdf
http://bigdata-madesimple.com/decoding-dimensionality-reduction-pca-and-svd/
https://jockchou.gitbooks.io/getting-started-with-mongodb/content/book/import.html
http://xxuan.me/2017-02-17-mongodb.html
http://www.open-open.com/lib/view/open1414223469247.html
https://www.cnblogs.com/rongyux/p/5602037.html
https://www.datacamp.com/courses/intro-to-python-for-data-science
http://blog.csdn.net/zm714981790/article/details/51304506
http://blog.csdn.net/sinat_26917383/article/details/53260117
http://blog.csdn.net/lily960427/article/details/78996500
https://github.com/jerry81333/SpamSMSFiltering
http://brandonrose.org/clustering_mobile#

In [70]:
len(user_data)

4524

In [71]:
w = open('user_data.txt', 'w', encoding='utf-8')
for x in user_data:
    if x:
        w.write(i)
        w.write('\n')
    else:
        continue
w.close()

In [3]:
# get words frequences greater than p 
import collections
def get_top_fre(words, p):
    fre = collections.Counter(words)
    fre = fre.most_common()
    fre = dict(fre)
    t1 = []
    t2 = []
    for k,v in fre.items():
        if v>p:
            t1.append(k)
            t2.append(v)
    #frequences = dict(zip(t1,t2))
    return t1

def get_count(words):
    fre = collections.Counter(words)
    return fre

In [4]:
#word cloud
from wordcloud import WordCloud
import matplotlib.pyplot as plt
def get_wordcloud(words):
    wordcloud = WordCloud(background_color="white", width=1000, height=860, margin=2).generate(str(words))
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.show()
    wordcloud.to_file('text.png')

In [24]:
from gensim import corpora, models, similarities
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

D:\Anaconda\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [28]:
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]
lsi = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=10)
index = similarities.MatrixSimilarity(lsi[corpus])

2018-07-23 21:34:34,595 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2018-07-23 21:34:34,598 : INFO : built Dictionary(815 unique tokens: ['analys', 'big', 'compon', 'dat', 'diagon']...) from 11 documents (total 939 corpus positions)
2018-07-23 21:34:34,600 : INFO : collecting document frequencies
2018-07-23 21:34:34,602 : INFO : PROGRESS: processing document #0
2018-07-23 21:34:34,604 : INFO : calculating IDF weights for 11 documents and 814 features (939 matrix non-zeros)
2018-07-23 21:34:34,607 : INFO : using serial LSI version on this node
2018-07-23 21:34:34,607 : INFO : updating model with new documents
2018-07-23 21:34:34,612 : INFO : preparing a new chunk of documents
2018-07-23 21:34:34,614 : INFO : using 100 extra samples and 2 power iterations
2018-07-23 21:34:34,615 : INFO : 1st phase: constructing (815, 110) action matrix
2018-07-23 21:34:34,623 : INFO : orthonormalizing (815, 110) action matrix
2018-07-23 21:34:34,668 : INFO : 2nd phase: running dense sv

In [31]:
test = texts[5]
bow = dictionary.doc2bow(test)
ml_lsi = lsi[bow]
sims = index[ml_lsi]
sort_sims = sorted(enumerate(sims), key=lambda item: -item[1])

In [43]:
lsi.print_topic(topicno=0)

'0.102*"footbal" + 0.102*"team" + 0.099*"learn" + 0.096*"world" + 0.094*"\'s" + 0.091*"word" + 0.091*"model" + 0.090*"hour" + 0.090*"vaccin" + 0.090*"ago"'